In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.request import Request, urlopen
from typing import List, Dict, Any

# Scrap the calendars of www.basketball-reference.com

In [3]:
head = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'fr;q=0.9,en-US,en;q=0.8',
      'Connection': 'keep-alive'}

Dictionnary that contains the months of every season and the last value of the list is the last day of the month for the regular season

In [4]:
season:Dict[int,List[Any]] = {
    2015 : ["october","november","december","january","february","march","april",15],
    2016 : ["october","november","december","january","february","march","april",13],
    2017 : ["october","november","december","january","february","march","april",12],
    2018 : ["october","november","december","january","february","march","april",11],
    2019 : ["october","november","december","january","february","march","april",10],
    2020 : ["october","november","december","january","february","march","july","august",14],
    2021 : ["december","january","february","march","april","may",16],
    2022 : ["october","november","december","january","february","march","april",10],
    2023 : ["october","november","december","january","february","march","april",9],
    2024 : ["october","november","december","january","february","march","april",14],
    2025 : ["october","november","december","january","february","march","april",13]
}

In [5]:
def scrap_code(year:int, month:str) -> pd.DataFrame:
    r = Request(url=f"https://www.basketball-reference.com/leagues/NBA_{str(year)}_games-{month}.html", headers = head)
    webpage = urlopen(r).read()
    bs = BeautifulSoup(webpage, "html.parser")

    if month == season[year][-2]:
        csk_values = [th.get("csk") for th in bs.select('th[data-stat="date_game"][csk]') if int(th.get("csk")[6:8]) <= season[year][-1]]
    else :
        csk_values = [th.get("csk") for th in bs.select('th[data-stat="date_game"][csk]')]
        
    # We get the Overtime mention to suppress them from the future game scrap     
    OT_values = [text.text for text in bs.select('td[data-stat="overtimes"]')][:len(csk_values)]

    df = pd.DataFrame({"game_codes":csk_values,"Overtime":OT_values,"season":year})
    
    return df

In [6]:
df = pd.DataFrame(columns=["game_codes","Overtime","season"])

In [8]:
for year in list(season.keys())[-1:]:
    for month in season[year][:-1]:
        df = pd.concat([df, scrap_code(year, month)])
        time.sleep(2)

In [9]:
df.to_excel("game_codes_2025.xlsx")